## Importacion del dataset

In [8]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-cp36-none-any.whl size=73265 sha256=c9b345dda99ba9e5a229abb811edb7d928f37d7a6531ec867cdc9e764f786da3
  Stored in directory: /root/.cache/pip/wheels/68/6d/9b/7a98271454edcba3b56328cbc78c037286e787d004c8afee71
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9


In [2]:
# Upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nexans","key":"d20f5011acd04d5155edf8f9f2c868ea"}'}

In [4]:
# Move kaggle.json
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
# Give permissions to kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
# Download dataset
! kaggle competitions download -c 'diabetic-retinopathy-detection'

 94% 77.6G/82.2G [26:24<01:34, 52.6MB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/kaggle/cli.py", line 67, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 758, in competition_download_cli
    quiet)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 725, in competition_download_files
    self.download_file(response, outfile, quiet)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 1599, in download_file
    out.write(data)
OSError: [Errno 28] No space left on device
